## 1. 싱글데이터 TR 요청

In [ ]:
from pykiwoom.kiwoom import *
import time
import pandas as pd

kiwoom = Kiwoom()
kiwoom.CommConnect(block=True)

In [ ]:
df = kiwoom.block_request("opt10001",
                          종목코드="005930",
                          output="주식기본정보",
                          next=0)

In [ ]:
df

In [ ]:
# 멀티데이터 TR
df = kiwoom.block_request("opt10081",
                          종목코드="005930",
                          기준일자="20200424",
                          수정주가구분=1,
                          output="주식일봉차트조회",
                          next=0)

In [ ]:
df.head()

In [6]:
# 연속조회
dfs = []
df = kiwoom.block_request("opt10081",
                          종목코드="005930",
                          기준일자="20200424",
                          수정주가구분=1,
                          output="주식일봉차트조회",
                          next=0)
dfs.append(df)

while kiwoom.tr_remained:
    df = kiwoom.block_request("opt10081",
                              종목코드="005930",
                              기준일자="20200424",
                              수정주가구분=1,
                              output="주식일봉차트조회",
                              next=2)
    dfs.append(df)
    time.sleep(1)

df = pd.concat(dfs)

In [7]:
df

,종목코드,현재가,거래량,거래대금,일자,시가,고가,저가,수정주가구분,수정비율,대업종구분,소업종구분,종목정보,수정주가이벤트,전일종가
0,005930,49350,15618347,771212,20200424,49650,49750,49000,,,,,,,
1,,49850,18754442,936174,20200423,50200,50300,49500,,,,,,,
2,,49850,18613864,912223,20200422,48700,50000,48350,,,,,,,
3,,49250,27407543,1347415,20200421,49400,49700,48700,,,,,,,
4,,50100,21866354,1106552,20200420,51400,51400,50000,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,,123,324837,1,19850109,126,126,122,,,,,,,
359,,127,845098,4,19850108,129,129,127,,,,,,,
360,,129,771895,3,19850107,129,130,128,,,,,,,
361,,128,108497,0,19850105,129,129,128,,,,,,,


## 2. 일봉 데이터 저장

In [ ]:
from pykiwoom.kiwoom import *
import datetime
import time

In [ ]:
# 로그인
kiwoom = Kiwoom()
kiwoom.CommConnect()

In [ ]:
kospi = kiwoom.GetCodeListByMarket("0")
kosdaq = kiwoom.GetCodeListByMarket("10")
codes = kospi + kosdaq

In [ ]:
now = datetime.datetime.now()
today = now.strftime("%Y%m%d")

In [ ]:
for i, code in enumerate(codes):
    print(f"{i}/{len(codes)} {code}")
    df = kiwoom.block_request("opt10081",
                              종목코드=code,
                              기준일자=today,
                              수정주가구분=1,
                              output="주식일봉차트조회",
                              next=0)
    out_name = "data/" + f"{code}.xlsx"
    df.to_excel(out_name)
    time.sleep(3.6)

In [8]:
import pandas as pd
import glob

xlsx_list = glob.glob("data/*.xlsx")

close_data = []

for xls in xlsx_list:
    code = xls.split(".")[0]
    df = pd.read_excel(xls)
    df2 = df[["일자", "현재가"]].copy()
    df2.rename(columns={"현재가": code}, inplace=True)
    df2 = df2.set_index("일자")
    df2 = df2[::-1]
    close_data.append(df2)
    
df = pd.concat(close_data, axis=1)
df.to_excel("merge.xlsx")


## 3. 모멘텀 전략 종목 선정 및 매수

In [9]:
import pandas as pd

df = pd.read_excel("merge.xlsx")
df["일자"] = pd.to_datetime(df["일자"], format="%Y%m%d")
df = df.set_index("일자")

In [10]:
# 20일 기준 수익률
return_df = df.pct_change(20)
return_df.tail()

,data\000020,data\000040,data\000050,data\000060,data\000070,data\000075,data\000080,data\000087,data\000100,data\000105,...,data\900340,data\950110,data\950130,data\950140,data\950160,data\950170,data\950190,data\950200,data\950210,data\950220
일자,,,,,,,,,,,,,,,,,,,,,
2022-07-25,0.111686,0.143084,0.052846,-0.036390,-0.021505,0.039792,-0.063091,-0.044503,0.045208,0.044444,...,-0.085185,0.128451,0.360000,0.047718,0.0,-0.059028,0.086420,0.082601,0.042918,0.129341
2022-07-26,0.094705,0.139683,0.048387,-0.052632,-0.013351,0.037801,-0.060937,-0.049738,0.028520,0.046296,...,-0.080292,0.127251,0.303249,0.055381,0.0,-0.074010,0.056886,0.082024,0.043860,0.159905
2022-07-27,0.092480,0.130366,0.048583,-0.037410,0.010959,0.039451,-0.032051,-0.039267,0.041293,0.036697,...,-0.099644,0.075561,0.287273,0.055328,0.0,-0.092282,0.047619,0.147368,0.063063,0.146572
2022-07-28,0.109391,0.159935,0.061224,0.024133,0.036466,0.030664,-0.021104,-0.031662,0.032143,0.040441,...,-0.095941,0.086651,0.212329,0.080043,0.0,-0.075342,-0.003021,0.170330,0.021834,0.148325
2022-07-29,0.100104,0.187602,0.105042,0.023739,0.066856,0.053913,-0.011438,-0.026525,0.032143,0.042357,...,-0.020080,0.082742,0.125000,0.141256,0.0,-0.032258,0.031348,0.220703,0.057778,0.230483


In [11]:
s = return_df.loc["2022-07-29"]
momentum_df = pd.DataFrame(s)
momentum_df.columns = ["모멘텀"]
momentum_df.head()

,모멘텀
data\000020,0.100104
data\000040,0.187602
data\000050,0.105042
data\000060,0.023739
data\000070,0.066856


In [12]:
momentum_df['순위'] = momentum_df['모멘텀'].rank(ascending=False)
momentum_df = momentum_df.sort_values(by="순위")
momentum_df[:10].to_excel("momentum_list.xlsx")

In [13]:
# 종목 불러와서 매수
df = pd.read_excel("momentum_list.xlsx")
df.columns = ["종목코드", "모멘텀", "순위"]

codes = df["종목코드"]
names = [kiwoom.GetMasterCodeName(code) for code in codes]
df["종목명"] = pd.Series(data=names)

In [14]:
df

,종목코드,모멘텀,순위,종목명
0,data\052020,1.136895,1,
1,data\530068,1.023517,2,
2,data\570045,1.019397,3,
3,data\500053,1.004100,4,
4,data\013720,0.933852,5,
5,data\258830,0.881639,6,
6,data\001570,0.818956,7,
7,data\247660,0.787654,8,
8,data\030960,0.757143,9,
9,data\115310,0.746914,10,


In [ ]:
accounts = kiwoom.GetLoginInfo("ACCNO")
account = accounts[0]

for code in codes:
    ret = kiwoom.SendOrder("시장가매수", "0101", account, 1, code, 100, 0, "03", "")
    time.sleep(0.2)
    print(code, "종목 시장가 주문 완료")